In [21]:
import torch
import torchvision
from torchvision import transforms
import numpy as np


import PIL
from PIL import Image
import torch.nn.functional as F

from diffusers import UNet2DModel
import tqdm
import glob


In [24]:
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Adjust these values as needed
])

model = UNet2DModel(
        sample_size=512,  # the target image resolution
        in_channels=2,  # the number of input channels, 3 for RGB images
        out_channels=1,  # the number of output channels
        layers_per_block=2,  # how many ResNet layers to use per UNet block
        block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channes for each UNet block
        down_block_types=( 
            "DownBlock2D",  # a regular ResNet downsampling block
            "DownBlock2D",
            "DownBlock2D", 
            "DownBlock2D", 
            "AttnDownBlock2D",
            "DownBlock2D",  # a ResNet downsampling block with spatial self-attention
        ), 
        up_block_types=(
            "UpBlock2D",  # a regular ResNet upsampling block
            "AttnUpBlock2D", # a ResNet upsampling block with spatial self-attention 
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D", 
            "UpBlock2D"
          ),
    )


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
image_path = '/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_2/images/Case_00001_0000_slice000.png'
image = Image.open(image_path).convert('L')


image_tensor = preprocess(image)
image_tensor = image_tensor.unsqueeze(0)

if image_tensor.shape[1] == 1:
    image_tensor = image_tensor.repeat(1, 2, 1, 1)

In [16]:
image_tensor.shape

torch.Size([1, 2, 512, 512])

In [17]:
model.eval()

model.to(device)
image_tensor = image_tensor.to(device)
with torch.no_grad():
    output = model(image_tensor,0).sample

torch.Size([1, 1, 512, 512])

In [ ]:
cosine_similarity = F.cosine_similarity(encoded_image1_flat.unsqueeze(0), encoded_image2_flat.unsqueeze(0))

print(f"Cosine similarity: {cosine_similarity.item()}")